In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns



plt.style.use('default')
sns.set_palette("husl")

# Configuración
config = {
    'output_path': '../data/processed/',
    'rul_threshold_alerta': 5000,  # km para alerta
    'rul_threshold_riesgo': [0, 5000, 10000, float('inf')]
}

# 1. CARGA DATOS ORIGINALES
print("📥 Cargando datos...")
df = pd.read_csv("../data/train_bogie_dataset.csv")
print(f"Shape original: {df.shape}")
print(f"Bogies únicos: {df['bogie_id'].nunique()}")
print("\nDistribución fallos originales:")
print(df['target_fault'].value_counts(normalize=True).round(4))

# 2. CREAR RUL POR BOGIE (clave del nuevo enfoque)
print("\n🔄 Calculando RUL...")
df = df.sort_values(['bogie_id', 'km'])  # Asegurar orden
df['km_total_bogie'] = df.groupby('bogie_id')['km'].transform('max')
df['RUL_km'] = df['km_total_bogie'] - df['km']

print(f"RUL creado. Stats:\n{df['RUL_km'].describe()}")

# 3. NUEVAS ETIQUETAS (menos desbalance)
df['is_alerta'] = (df['RUL_km'] < config['rul_threshold_alerta']).astype(int)
df['label_riesgo'] = pd.cut(df['RUL_km'], 
                           bins=config['rul_threshold_riesgo'], 
                           labels=[2,1,0]).astype(int)

print("\n🎯 Distribución nuevas etiquetas:")
print("is_alerta:", df['is_alerta'].value_counts(normalize=True).round(4))
print("label_riesgo:", df['label_riesgo'].value_counts().sort_index())

# 4. VISUALIZACIÓN RUL por bogie (diagnóstico)
fig, axes = plt.subplots(2, 2, figsize=(15,10))

# RUL por bogie
sample_bogies = df.groupby('bogie_id')['RUL_km'].max().nlargest(6).index
for bogie in sample_bogies:
    bogie_data = df[df['bogie_id']==bogie].sort_values('km')
    axes[0,0].plot(bogie_data['km'], bogie_data['RUL_km'], alpha=0.7)
axes[0,0].set_title('Evolución RUL (muestra bogies)')
axes[0,0].set_xlabel('km'); axes[0,0].set_ylabel('RUL (km)')

# Histograma RUL
axes[0,1].hist(df['RUL_km'], bins=50, alpha=0.7, edgecolor='black')
axes[0,1].axvline(config['rul_threshold_alerta'], color='red', lw=2, ls='--')
axes[0,1].set_title('Distribución RUL'); axes[0,1].set_xlabel('RUL (km)')

# Fallos originales vs nuevas alertas
df_plot = df.groupby('bogie_id').agg({
    'target_fault': 'sum',
    'is_alerta': 'sum',
    'RUL_km': 'min'
}).reset_index()
axes[1,0].scatter(df_plot['target_fault'], df_plot['is_alerta'], alpha=0.6)
axes[1,0].set_xlabel('Fallos originales'); axes[1,0].set_ylabel('Alertas RUL')
axes[1,0].set_title('Comparación fallos vs alertas')

# Distribución por riesgo
df['label_riesgo'].value_counts().sort_index().plot(kind='bar', ax=axes[1,1])
axes[1,1].set_title('Distribución riesgo (0=bajo, 2=alto)')

plt.tight_layout()
plt.savefig('../reports/figures/01_rul_overview.png', dpi=300, bbox_inches='tight')
plt.show()

# 5. GUARDAR DATASET PROCESADO

df.to_csv(f"{config['output_path']}bogies_rul_procesado.csv", index=False)
print(f"\n💾 Dataset guardado: {config['output_path']}bogies_rul_procesado.csv")

# 6. RESUMEN FINAL
print("\n📊 RESUMEN LIMPIADO:")
print(f"Filas: {len(df):,}")
print(f"Features candidatas: {len(df.columns)-6}")  # Excluyendo id,timestamp,RULs
print(f"Balance alerta: {df['is_alerta'].mean():.1%} ← ¡MUCHO MEJOR!")
print("\n✅ Listo para modelado en 02_modelos_rul.ipynb")


📥 Cargando datos...
Shape original: (200000, 23)
Bogies únicos: 40

Distribución fallos originales:
target_fault
0.0    0.985
1.0    0.015
Name: proportion, dtype: float64

🔄 Calculando RUL...


KeyError: 'km'